# Load Libraries

In [74]:
import json, requests
import pandas as pd

from io import StringIO
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

# Set Up Desired Capabilities

In [2]:
options = ChromeOptions()
options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})

# Tangkap Network

In [3]:
def process_browser_log_entry(entry):
    response = json.loads(entry['message'])['message']
    return response

# Buka Browser

In [4]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

#driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
#driver = webdriver.Chrome()

# Buka Halaman Target

In [5]:
driver.get('https://ojk.go.id/id/kanal/perbankan/data-dan-statistik/laporan-keuangan-perbankan/Default.aspx')

# Pindah ke iframe

In [6]:
# devframe
driver.switch_to.frame(driver.find_element(By.ID, 'devframe'))

In [56]:
# MainReportFrame

# Pilih Unit Usaha Syariah

In [7]:
#unit_usaha = driver.find_element(By.XPATH, '//span[contains(text(), "Unit Usaha Syariah")]')
unit_usaha = driver.find_element(By.XPATH, '//span[contains(text(), "Bank Umum Syariah")]')

#unit_usaha = driver.find_element(By.XPATH, '//a[@id="tab-1012"]')
unit_usaha.click()

# Pakai requests

In [8]:
S = requests.Session()
S.cookies.update({c['name']: c['value'] for c in driver.get_cookies()})

In [9]:
r = S.get('https://ojk.go.id/id/kanal/perbankan/data-dan-statistik/laporan-keuangan-perbankan/Default.aspx')

In [10]:
r.text

'\r\n<!DOCTYPE html>\r\n<html lang="en" dir="ltr" class="ms-isBot">\r\n    <head><meta http-equiv="X-UA-Compatible" content="IE=10" /><meta name="viewport" content="width=device-width, initial-scale=1.0" /><meta name="description" /><meta name="author" /><meta http-equiv="Content-type" content="text/html; charset=utf-8" /><meta http-equiv="Expires" content="0" /><link rel="shortcut icon" href="/Style%20Library/ojk/img/favicon.ico" type="image/x-icon" /><link rel="icon" href="/Style%20Library/ojk/img/favicon.ico" type="image/x-icon" /><title>\r\n\tLaporan Keuangan Perbankan\r\n</title><link id="CssLink-abebceb4b6c941788a104cc10fb2984f" rel="stylesheet" type="text/css" href="/_layouts/15/1033/styles/Themable/corev15.css?rev=BiOgxuWcxBnhKNutTFJcDA%3D%3DTAG0"/>\n<style id="SPThemeHideForms" type="text/css">body {opacity:0 !important}</style><link href="/SiteAssets/css/bootstrap.min.css" rel="stylesheet" /><link href="/SiteAssets/css/bootstrap-custom.css" rel="stylesheet" /><link href="/Sty

# Post

In [14]:
r = S.get('https://cfs.ojk.go.id/cfs/Report.aspx?BankTypeCode=BUS&BankTypeName=Bank+Umum+Syariah')

In [17]:
#r.text
viewstate = r.text.split('id="__VIEWSTATE" value="')[1].split('"')[0]
viewstategenerator = r.text.split('id="__VIEWSTATEGENERATOR" value="')[1].split('"')[0]
eventvalidation = r.text.split('id="__EVENTVALIDATION" value="')[1].split('"')[0]

In [18]:
payload = {
    'submitDirectEventConfig': '{"config":{"extraParams":{"node":"root"}}}',
    '__EVENTTARGET': 'DefaultResourceManager',
    '__EVENTARGUMENT': 'BankTreeStore|postback|read',
    '__VIEWSTATE': viewstate,
    '__VIEWSTATEGENERATOR': viewstategenerator,
    '__EVENTVALIDATION': eventvalidation,
    'ojk.cfs.PeriodRadioGroup_Group': 'B',
    'Month': 'Agustus',
    '_Month_state': '[{"value":"8","text":"Agustus","index":7}]',
    'Year': '2024',
    'BankCode': '',
    'ReportTree_SM': '',
    'ReportTree_CheckNodes': '',
}

In [21]:
headers = {
    'X-Ext.Net': 'delta=true',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'
}

In [22]:
r = S.post('https://cfs.ojk.go.id/cfs/Report.aspx?BankTypeCode=BUS&BankTypeName=Bank+Umum+Syariah', data = payload, headers=headers)

In [23]:
r.text

'{script:"ojk.cfs.Year.setMinValue(2000.0);ojk.cfs.Year.setMaxValue(2024.0);ojk.cfs.ParameterPanel.getForm().checkValidity();ojk.cfs.PeriodRadioGroup.setWidth(300);ojk.cfs.PeriodRadioGroup.setFieldLabel(\\"Jangka Waktu\\");ojk.cfs.ReportPeriodContainer.setFieldLabel(\\"Periode Pelaporan\\");ojk.cfs.ReportLabel.setValue(\\"Laporan\\");",serviceResponse:{success:true,data:{"children":[{"id":"BUS.02","text":"BUS.02-BANK PEMERINTAH DAERAH","children":[{"id":"116","leaf":true,"iconCls":X.net.RM.getIcon("ApplicationAdd"),"text":"116-PT BANK ACEH"},{"id":"128","leaf":true,"iconCls":X.net.RM.getIcon("ApplicationAdd"),"text":"128-PT BANK NTB SYARIAH"}]},{"id":"BUS.03","text":"BUS.03-BANK SWASTA NASIONAL","children":[{"id":"147","leaf":true,"iconCls":X.net.RM.getIcon("ApplicationAdd"),"text":"147-PT BANK MUAMALAT INDONESIA"},{"id":"253","leaf":true,"iconCls":X.net.RM.getIcon("ApplicationAdd"),"text":"253-PT Bank Nano Syariah"},{"id":"405","leaf":true,"iconCls":X.net.RM.getIcon("ApplicationAdd"),

In [29]:
r_data = r.text.split('data:')[1].replace(',"iconCls":X.net.RM.getIcon("ApplicationAdd")', '')

In [45]:
r_data[12:-3]

'[{"id":"BUS.02","text":"BUS.02-BANK PEMERINTAH DAERAH","children":[{"id":"116","leaf":true,"text":"116-PT BANK ACEH"},{"id":"128","leaf":true,"text":"128-PT BANK NTB SYARIAH"}]},{"id":"BUS.03","text":"BUS.03-BANK SWASTA NASIONAL","children":[{"id":"147","leaf":true,"text":"147-PT BANK MUAMALAT INDONESIA"},{"id":"253","leaf":true,"text":"253-PT Bank Nano Syariah"},{"id":"405","leaf":true,"text":"405-PT. Bank Victoria Syariah"},{"id":"422","leaf":true,"text":"422-PT. Bank BRI Syariah"},{"id":"425","leaf":true,"text":"425-PT. Bank Jabar Banten Syariah"},{"id":"427","leaf":true,"text":"427-PT. Bank BNI Syariah"},{"id":"451","leaf":true,"text":"451-PT BANK SYARIAH INDONESIA"},{"id":"506","leaf":true,"text":"506-PT BANK MEGA SYARIAH"},{"id":"517","leaf":true,"text":"517-PT Bank Panin Dubai Syariah Tbk"},{"id":"521","leaf":true,"text":"521-PT BANK KB BUKOPIN SYARIAH"},{"id":"536","leaf":true,"text":"536-PT. Bank BCA Syariah"},{"id":"547","leaf":true,"text":"547-PT BANK BTPN SYARIAH TBK"},{"i

In [46]:
dd = json.loads(r_data[12:-3])

In [47]:
dd

[{'id': 'BUS.02',
  'text': 'BUS.02-BANK PEMERINTAH DAERAH',
  'children': [{'id': '116', 'leaf': True, 'text': '116-PT BANK ACEH'},
   {'id': '128', 'leaf': True, 'text': '128-PT BANK NTB SYARIAH'}]},
 {'id': 'BUS.03',
  'text': 'BUS.03-BANK SWASTA NASIONAL',
  'children': [{'id': '147',
    'leaf': True,
    'text': '147-PT BANK MUAMALAT INDONESIA'},
   {'id': '253', 'leaf': True, 'text': '253-PT Bank Nano Syariah'},
   {'id': '405', 'leaf': True, 'text': '405-PT. Bank Victoria Syariah'},
   {'id': '422', 'leaf': True, 'text': '422-PT. Bank BRI Syariah'},
   {'id': '425', 'leaf': True, 'text': '425-PT. Bank Jabar Banten Syariah'},
   {'id': '427', 'leaf': True, 'text': '427-PT. Bank BNI Syariah'},
   {'id': '451', 'leaf': True, 'text': '451-PT BANK SYARIAH INDONESIA'},
   {'id': '506', 'leaf': True, 'text': '506-PT BANK MEGA SYARIAH'},
   {'id': '517', 'leaf': True, 'text': '517-PT Bank Panin Dubai Syariah Tbk'},
   {'id': '521', 'leaf': True, 'text': '521-PT BANK KB BUKOPIN SYARIAH'

In [51]:
bank_ls = []

for d in dd:
    for child in d['children']:
        bank_ls.append({'id': child['id'], 'text': child['text']})

In [52]:
bank_ls

[{'id': '116', 'text': '116-PT BANK ACEH'},
 {'id': '128', 'text': '128-PT BANK NTB SYARIAH'},
 {'id': '147', 'text': '147-PT BANK MUAMALAT INDONESIA'},
 {'id': '253', 'text': '253-PT Bank Nano Syariah'},
 {'id': '405', 'text': '405-PT. Bank Victoria Syariah'},
 {'id': '422', 'text': '422-PT. Bank BRI Syariah'},
 {'id': '425', 'text': '425-PT. Bank Jabar Banten Syariah'},
 {'id': '427', 'text': '427-PT. Bank BNI Syariah'},
 {'id': '451', 'text': '451-PT BANK SYARIAH INDONESIA'},
 {'id': '506', 'text': '506-PT BANK MEGA SYARIAH'},
 {'id': '517', 'text': '517-PT Bank Panin Dubai Syariah Tbk'},
 {'id': '521', 'text': '521-PT BANK KB BUKOPIN SYARIAH'},
 {'id': '536', 'text': '536-PT. Bank BCA Syariah'},
 {'id': '547', 'text': '547-PT BANK BTPN SYARIAH TBK'},
 {'id': '601', 'text': '601-Bank Perkreditan Rakyat Syariah (BPRS)'},
 {'id': '947', 'text': '947-PT BANK ALADIN SYARIAH TBK'}]

In [54]:
#url_neraca = 'https://cfs.ojk.go.id/cfs/ReportViewerForm.aspx?BankCodeNumber=116&BankCode=PT+BANK+ACEH&Month=1&Year=2024&FinancialReportPeriodTypeCode=B&FinancialReportTypeCode=BPS-906-00031'
url_neraca_1 = 'https://cfs.ojk.go.id/cfs/ReportViewerForm.aspx?BankCodeNumber='
url_neraca_2 = '&BankCode='
url_neraca_3 = '&Month='
url_neraca_4 = '&Year='
url_neraca_5 = '&FinancialReportPeriodTypeCode=B&FinancialReportTypeCode=BPS-906-00031'


## Requests get Table

In [85]:
for bank in bank_ls[:1]:
    url = url_neraca_1 + bank['id'] + url_neraca_2 + bank['text'].split('-')[1].replace(' ', '+') + url_neraca_3 + '1' + url_neraca_4 + '2024' + url_neraca_5
    r = S.get(url)
    #print(r.text)
    tables = pd.read_html(StringIO(r.text))

    for table in tables:
        if 'Pos-pos' in table.values:
            print(table)

     0    1                               2                      3
0  NaN  NaN                             NaN                    NaN
1  NaN  NaN                             NaN  (dalam Jutaan Rupiah)
2  NaN   No                         Pos-pos           Januari 2024
3  NaN   10                            ASET                    NaN
4  NaN   20                          1. Kas                1214465
..  ..  ...                             ...                    ...
77 NaN  750             a. Tahun-tahun lalu                 697729
78 NaN  760               b. Tahun berjalan                  32970
79 NaN  770  c. Dividen yang dibayarkan -/-                      0
80 NaN  780                   TOTAL EKUITAS                3824743
81 NaN  790    TOTAL LIABILITAS DAN EKUITAS               27090373

[82 rows x 4 columns]


: 

In [82]:
tables[4]

,0
0,Laporan Publikasi BulananNeraca31 Januari 2024...


In [75]:
for bank in bank_ls[:]:
    url = url_neraca_1 + bank['id'] + url_neraca_2 + bank['text'].split('-')[1].replace(' ', '+') + url_neraca_3 + '1' + url_neraca_4 + '2024' + url_neraca_5
    driver.get(url)
    try:
        table = driver.find_element(By.XPATH, '//div[contains(text(), "Pos-pos")]/parent::td/parent::tr/parent::tbody/parent::table')
        tb = pd.read_html('<table>' + StringIO(str(table.get_attribute('innerHTML'))) + '</table>')
        tb[0].iloc[:, -2:].to_excel(bank['text'].replace(' ', '_') + '.xlsx', index=False)
    except: True

In [66]:
tb = pd.read_html('<table>' + table.get_attribute('innerHTML') + '</table>')

C:\Users\aansubarkah\AppData\Local\Temp\ipykernel_14904\3811163334.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tb = pd.read_html('<table>' + table.get_attribute('innerHTML') + '</table>')


In [71]:
tb[0].iloc[:, -2:]

,2,3
0,NaN,NaN
1,NaN,(dalam Jutaan Rupiah)
2,Pos-pos,Januari 2024
3,ASET,NaN
4,1. Kas,1214465
...,...,...
77,a. Tahun-tahun lalu,697729
78,b. Tahun berjalan,32970
79,c. Dividen yang dibayarkan -/-,0
80,TOTAL EKUITAS,3824743


# Pilih Jangka Waktu

In [156]:
# devframe
driver.switch_to.frame(driver.find_element(By.ID, 'MainReportFrame'))

In [113]:
triwulanan = driver.find_element(By.XPATH, '//label[contains(text(), "Triwulanan")]/preceding-sibling::input')

In [114]:
triwulanan.click()

In [9]:
#semesteran = driver.find_element(By.XPATH, '//label[contains(text(), "Semesteran")]/preceding-sibling::input')
#semesteran.click()

# Pilih Bulan

## Klik untuk memunculkan daftar nama bulan

In [10]:
driver.find_element(By.ID, 'ext-gen1060').click()

## Pilih nama bulan

In [11]:
periode_bulan = driver.find_element(By.XPATH, '//li[@role="option" and contains(text(), "Juni")]')
periode_bulan.click()

In [20]:
bulan_ls = ['Maret', 'Juni', 'September', 'Desember']
for bulan in bulan_ls:
    driver.find_element(By.ID, 'ext-gen1060').click()
    periode_bulan = driver.find_element(By.XPATH, '//li[@role="option" and contains(text(), "' + bulan + '")]')
    periode_bulan.click()

# Pilih Tahun

Menggunakan sedikit algoritma.

Defaultnya tahun dipilih adalah tahun sekarang (2024).
- Tahun terbesar = tahun sekarang
- Tahun terkecil = 2000

Algoritma digunakan:
- Klik tombol bawah, sebanyak tahun yang dituju

In [38]:
#periode_tahun = driver.find_element(By.XPATH, '//input[@id="Year-inputEl"]')
#periode_tahun.send_keys('')
#periode_tahun.send_keys('2021')
tahun_down = driver.find_element(By.XPATH, '//div[contains(@class, "x-form-spinner-down")]')
#tahun_down = driver.find_element(By.XPATH, '//div[@id="ext-gen1066"]')
tahun_down.click()

# Pilih Bank

## Munculkan Daftar Bank

In [157]:
driver.find_element(By.XPATH, '//table[@id="BankCode-triggerWrap"]').click()

In [158]:
browser_log = driver.get_log('performance')

In [159]:
events = [process_browser_log_entry(entry) for entry in browser_log]
events = [event for event in events if 'Network.response' in event['method']]

In [160]:
for e in events:
    if 'response' in e['params'] and 'headers' in e['params']['response'] and 'content-type' in e['params']['response']['headers']:# and: 'text/plain' in e['params']['response']['headers']['content-type']:
        #print(e['params']['response']['headers']['content-type'])
        print(e)

{'method': 'Network.responseReceived', 'params': {'frameId': '037BE65E7C7E8BF4D43230DF281965E0', 'hasExtraInfo': True, 'loaderId': '68A83531B3926EAF5E0D03F9D3090F6A', 'requestId': '23128.504', 'response': {'alternateProtocolUsage': 'unspecifiedReason', 'charset': '', 'connectionId': 220, 'connectionReused': False, 'encodedDataLength': 1736, 'fromDiskCache': False, 'fromPrefetchCache': False, 'fromServiceWorker': False, 'headers': {'access-control-allow-methods': 'GET, HEAD, PUT, POST, DELETE', 'access-control-allow-origin': '*', 'access-control-expose-headers': 'Content-Range, Content-Length, ETag, Content-Type', 'access-control-max-age': '3000', 'age': '309', 'cache-control': 'max-age=3600, public', 'content-encoding': 'gzip', 'content-type': 'application/javascript', 'date': 'Mon, 02 Sep 2024 03:54:35 GMT', 'etag': 'W/"3953117e37c726b61b5619804e7b8955"', 'last-modified': 'Wed, 28 Aug 2024 12:55:44 GMT', 'server': 'CDN77-Turbo', 'vary': 'Accept-Encoding', 'via': '1.1 f0ff3515536254a60

## Pilih BPD

In [89]:
driver.find_element(By.XPATH, '//span[contains(text(), "BANK PEMERINTAH DAERAH")]/preceding-sibling::img[2]').click()

## Daftar Nama BPD

In [82]:
banks = driver.find_elements(By.XPATH, '//tr[contains(@id,"record-UUS.02")]/following-sibling::tr')

In [92]:
for b in banks:
    print(b.text)
    b.click()

UUS.03-BANK SWASTA NASIONAL


In [ ]:
#https://cfs.ojk.go.id/cfs/ReportViewerForm.aspx?BankCodeNumber=111&BankCode=PT+BPD+DKI+-+UUS&Month=6&Year=2023&FinancialReportPeriodTypeCode=R&FinancialReportTypeCode=UUS-906-00044A
#https://cfs.ojk.go.id/cfs/ReportViewerForm.aspx?BankCodeNumber=002&BankCode=PT+BANK+RAKYAT+INDONESIA+(PERSERO)%2c+Tbk&Month=1&Year=2024&FinancialReportPeriodTypeCode=B&FinancialReportTypeCode=PGWS-908-00031
#https://cfs.ojk.go.id/cfs/ReportViewerForm.aspx?BankCodeNumber=147&BankCode=PT+BANK+MUAMALAT+INDONESIA&Month=1&Year=2024&FinancialReportPeriodTypeCode=B&FinancialReportTypeCode=BPS-906-00031
#https://cfs.ojk.go.id/cfs/ReportViewerForm.aspx?BankCodeNumber=116&BankCode=PT+BANK+ACEH&Month=1&Year=2024&FinancialReportPeriodTypeCode=B&FinancialReportTypeCode=BPS-906-00031

In [ ]:
#https://cfs.ojk.go.id/cfs/Report.aspx?BankTypeCode=BUS&BankTypeName=Bank+Umum+Syariah

In [53]:
driver.get('https://cfs.ojk.go.id/cfs/ReportViewerForm.aspx?BankCodeNumber=116&BankCode=PT+BANK+ACEH&Month=1&Year=2024&FinancialReportPeriodTypeCode=B&FinancialReportTypeCode=BPS-906-00031')